#  **Photolithography** 

### __Before you start the notebook please run the import section bellow.__

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
import math
from IPython.display import display, Markdown, clear_output
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, widgets, fixed, Dropdown, Output, IntSlider
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from matplotlib.widgets import Slider
from scipy.fftpack import fft2, ifft2, fftshift
from ipywidgets import interact
import ipywidgets as widgets
from base64 import b64encode
from IPython.display import display, HTML

### __Spin Coating__

**Spin coating** is a widely used technique in microfabrication processes that enables the deposition of thin films with precise control over their thickness and uniformity. It is a simple and efficient method that involves spinning a liquid or viscous material onto a substrate, which spreads and forms a thin film due to centrifugal forces. Note that before spin coating, a surface cleaning step can performed to remove contaminants, particles, and organic residues that could affect the quality of subsequent coatings or patterning steps



In [ ]:
mp4 = open('spincoating_clip.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=1000 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

### *Thickness/rotation speed curves*




We study here 2 resists used in EPFL's CMi, the ["AZ 1512 HS"](https://www.epfl.ch/research/facilities/cmi/process/photolithography/photoresist-selection/az-1512-hs/) and the lift-off resist (LOR) ["LOR 5A"](https://www.epfl.ch/research/facilities/cmi/process/photolithography/photoresist-selection/lor-5a/).

We will generate curves detailing thickness T [$\mu m$] in function of the rotation speed $\omega$ [$rpm$]. Practically, recipes would start with a short spreading step at a set speed (for example, 500 RPM for 5 seconds) and then a longer main coating step.

For the resists mentionned above, the relation between the two variables T and $\omega$ is simplified as :
\begin{equation}
T = \alpha \cdot \omega^{-\beta} [\mu m]
\end{equation}

The next cell will load two datasets of measured RPM and achieved thickness for these two commonly used resists. Your task is to find the constants $\alpha$ and $\beta$ for that process. For this, you’re provided with sliders that will impact the production of the green curve representing the scaling law aforementioned. Note the pairs $\alpha,\beta$ that best match the two datasets.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive

# Given RPM-Thickness data
omega_resist = np.array([1000, 2000, 3000, 4000, 5000, 6000])
T_resist = np.array([2.61, 1.82, 1.47, 1.27, 1.13, 1.03])
omega_LOR = np.array([1000, 2000, 3000, 4000, 5000, 6000])
T_LOR = np.array([1.01, 0.71, 0.58, 0.50, 0.45, 0.41])

# x vector for curve fitting
x_values = np.linspace(1000, 6000, 1000)

# Function to update plot
def update_plot(alpha, beta):
    plt.figure(figsize=(10, 6))
    
    # Plotting the given data points
    plt.plot(omega_resist, T_resist, 'ro', label='AZ1215 HS Data Points')
    plt.plot(omega_LOR, T_LOR, 'bo', label='LOR 5A Data Points')
    
    # Calculating the curve y = alpha * x^-beta
    y_values = alpha * x_values ** -beta
    
    # Plotting the curve
    plt.plot(x_values, y_values, 'g-', label=f'Curve: $y = {alpha} \cdot x^{{-{round(beta, 3)}}}$')
    
    # Setting labels and title
    plt.xlabel('RPM')
    plt.ylabel('Thickness')
    plt.title('Interactive Plot of RPM vs Thickness with Curve Fitting')
    plt.grid(True)
    plt.legend()
    plt.show()

# Creating sliders for alpha and beta
interactive_plot = interactive(update_plot, alpha=(1, 100, 1), beta=(0.01, 1.5, 0.01))

# Displaying the interactive plot with sliders
interactive_plot

Which of these parameters is more dependent on the resin type? Why? (Double click to edit the cell and add your answer).

`Now that you know` $\alpha$ `and` $\beta$ `for AZ 1512 HS, calculate the value of rotation speed that is required to obtain 1.5 microns resist thickness.`

In [ ]:
TT = 1.5 #microns 
omega = 0 #= ...   # write the formula to find omega from T. Hint: The variables alpha and beta will be read from the sliders.
print('The required rotation speed for ', TT, ' micrometer thickness is ', omega, ' RPM')

### __Exposure and development__

**Photolithography** is a fundamental technique widely employed in microfabrication processes to pattern and define complex features on substrates with high precision. It is a key process in the production of microelectronic devices, integrated circuits, microfluidic systems, and various other microscale structures.

It is based on the principle of electromagnetic interaction and the modification of a resist material through the exposure to photons or electrons. The resist material is a photosensitive material that undergoes a chemical or physical change upon exposure. The patterned resist acts as a mask for subsequent processes, defining regions that will either be selectively etched or where materials will be deposited.

The **lithography** process begins with the **design of the desired pattern**, typically using computer-aided design (CAD) software. The design is then transferred to a photomask, which contains the pattern as opaque and transparent regions. The mask serves as a template for pattern transfer onto the resist-coated substrate.

During **exposure**, the resist is here subjected to UV light ; the photons or electrons interact with the resist, inducing chemical reactions or physical changes in the material. This modification of the resist's properties enables the creation of a pattern corresponding to the desired design.

Following exposure, the resist undergoes a **development** step. Development involves treating the resist with a developer solution that selectively removes either the exposed since here we are using a positive resist. This step reveals the patterned resist on the substrate, ready for subsequent processing step.



In [ ]:
mp4 = open('litho_clip.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=1000 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In this video, we observe several procedures. First, we see a typical 5-inch quartz mask (a square glass plate with a chromium layer) and how this mask is exposed in the direct laser writing machine VPG200 at CMi. Later on, we view a glass wafer on which a metal pattern is created using lift-off, utilizing the previously displayed mask. The operator employs a mask aligner to align the pattern on the mask with the wafer before exposing it to light through the mask (akin to a stencil). Finally, after the metal deposition by evaporation (which is not depicted in the video), the wafer is immersed in a solvent to remove the remaining resist and the metal on top of it. Only the metal deposited on the developed resist regions remains.

# *Exercice Section*
* ### Dose of exposure

To achieve proper development results, it is important to choose the exposure dose carefully. Before diving into further details, let's analyze the recommended dose for a specific thickness of AZ 1512 HS resist on the MABA6 and MA6 Gen3 machines. Please be aware that the presented curve is specific to this particular case and is provided to give you an idea of the potential behavior you might encounter.

`Run the following cell to display the recommended doses for a given thickness`

In [ ]:
plt.close('all')
# Determine the number of points and the values of doses, for the interpolation

def dev_curve(dose):
    thick=-3.08555*10**-10*dose**6 + 7.88354*10**-8*dose**5-8.17055*10**-6*dose**4+0.000437742*dose**3-0.012214*dose**2+0.173364*dose+0
    return thick

dose = np.arange(0, 80, 1, float) 
thickness= dev_curve(dose)

# Plot
def update(x_index):
    x_val = dose[x_index]
    y_val = thickness[x_index]

    fig, ax = plt.subplots()
    ax.plot(dose, thickness)
    ax.set_title('Dose/thickness removed curve')
    ax.set_ylabel('Thickness [um]')
    ax.set_xlabel('Dose [mJ/cm^2]')

    ax.axvline(x=x_val, color='r', linestyle='--', alpha = 0.3)
    ax.axhline(y=y_val, color='r', linestyle='--', alpha = 0.3)
    ax.text(x_val, y_val, "Dose =" + str(round(x_val, 3)) + r" $[mJ/cm^2]$,"+ "\nThickness =" + str(round(y_val, 3)) + r" [$\mu m]$",
            va='center', ha='center', fontsize = 'small', alpha=0.7)
    plt.show()


interact(update, 
         x_index=widgets.IntSlider(min=0, max=len(dose)-1, step=1, value=0))

`We used here a discrete slider over 80 values, since this is what we used to draw the function. What about the points in between? How would you read the exact value for a certain thickness? Either try editing the code in the next cell to get all your values using interpolation, or write an approximation by pen and paper, using the nearest neighors. `


In [ ]:
# Create the interpolation function, by filling in the comment
f =interpolate.interp1d(x= np.zeros(10) , y= np.zeros(10),             #what are we interpolating?, fill in x and y 
                        kind='linear', fill_value="extrapolate")
# Thickness values from the table
table_thickness = np.array([500, 700, 1000, 1500, 2000])

required_doses = f(table_thickness)

df = pd.DataFrame(data = required_doses, index=table_thickness, columns= ["Required dose [mJ/cm^2]"]).rename_axis('Thickness [nm]')
display(df)

* # Exposure patterns simulation

`This part of the notebook shows an example for a certain mask drawn in KLayout, choosing other variables!`

`In this part of the exercise, We will work with a sample mask that was created using the software KLayout (editor version).`

A mask is given by geometrical entities encoded in a special digital format. The most common of these is the binary format GDS. Given the complexity of this format, here we are going to use a simple representation of a closed polygon as a set of x:y points in a text file. The series of points are the vertices of the polygon, the first and last point have to be identical (closed shape) and there has to be non-intersecting segments and a clear inside region defined. The values of the points are integers in nm units. As an example here we provide a few text files: 'mask_ThreeFingers.txt', 'mask_ThreeTriangFingers.txt', 'mask_COMB_200nm.txt', 'mask_COMB_300nm.txt' and 'mask_COMB_500nm.txt'. When you run the following code you will immediately understand what polygons have been encoded in these files. For now, let us focus on the first file. The mask file is imported by the code (see further). 

When a patterned mask is introduced in the light's path, it results in a non-uniform exposure on the resist. However, due to light's wavelike nature, it forms diffraction patterns. We have used a far-field approximation combined with Fourier transforms to model this effect. See the corresponding lines of code where fft (fast Fourier transform) appears to understand what's going on.

Upon running the code, three distinct figures will be generated, depending on the light source that you chose. 

The first figure depicts the actual local dose accumulated on the resist during the duration of the exposure. This quantity will determine the developed thickness (as we learnt in the previous seciton). It is a surface plot of dose vs. x and y (3D representation).

To simplify we have included a slider to selecte a `y=y_pixel` plane cut. From this choice we can obtaine a line plot containing: 
* the dose profile along x on that y coordinate (blue)
* the threshold value necessary for full development (red dashed), 
* and the development function represented as a binary function (orange) indicating whether a zone is full-thickness developed or not developed completely.

Finally, the last two figures illustrate the original intended mask and the aerial image of the developed areas (white = developed, black = not fully developed). 

`Feel free to execute the code and explore the various parameters!`

`Fill in the TH value in the parameters with the found dose to expose your resist. TH here represents the threshold above which the resist is fully developed, and below which it isn't developed at all (this is an approximation, as you might know about grayscale lithography).`

In [ ]:
# Parameters
global N
N = int(600)    #number of points on the x and y scales
init_th=1.5     #microns
TH = 30         # Threshold intensity ### ENTER THE VALUE THAT YOU FOUND IN THE PREVIOUS STEP
NA = 1.0
# Numerical aperature
global prev_value
prev_value = 'ArF excimer laser'
 
# Reading mask shape from file
with open("mask_ThreeFingers.txt", "r") as file:
#with open("mask_ThreeTriangFingers.txt", "r") as file:
#with open("mask_COMB-200nm.txt", "r") as file:
#with open("mask_COMB-300nm.txt", "r") as file:
#with open("mask_COMB-500nm.txt", "r") as file:
    data = file.readlines()
X = []
Y = []
for line in data:
    line = line.strip().split(":")
    X.append(int(line[0])/1000.)
    Y.append(int(line[1])/1000.)
X = X-(np.max(X)+np.min(X))/2
Y = Y-(np.max(Y)+np.min(Y))/2
Lxy=np.max([np.max(X)-np.min(X),np.max(Y)-np.min(Y)]) #this is done so that the domain is square
margin_factor=1.2         # a margin will be left around the mask image
Lx = margin_factor*Lxy    # Mask real dimensions (um), dimension increased to have the black area around the mask
Ly = margin_factor*Lxy    # Mask real dimensions (um)
dx = Lx/N
dy = Ly/N
 
#we first have a function that calculates the dose profile
#it will convert the mask polygon into a pixel field with 1's and 0'2 according to design
#then if will apply the fft model to calculate dose.
#finally, by comparing with the threshold, an aerial view of the developed regions is also calculated.
def update_3D(lamda,N, t_exp_sec):
    # Center the polygon
    x = np.linspace(-margin_factor/2*Lxy+dx/2,margin_factor/2*Lxy-dx/2,N)
    y = np.linspace(-margin_factor/2*Lxy+dy/2,margin_factor/2*Lxy-dy/2,N)
    # Create mask matrix
    x,y = np.meshgrid(x,y)
    x = np.reshape(x,-1)
    y = np.reshape(y,-1)
    mask = Path(np.column_stack((X,Y)))
    mask = mask.contains_points(np.column_stack((x,y)))
    mask = np.reshape(mask,(N,N))
    mask = mask.astype(float) # this variable now containes a matrix of 0's and 1's for points outside or inside the design, respectively
    # Calculations creating different domains
    
    nx, ny = np.meshgrid(np.arange(-N/2+1,N/2+1),np.arange(-N/2+1,N/2+1))
    fx = (1/dx)*(1/N)*nx
    fy = (1/dy)*(1/N)*ny
    # Discrete frequency domain (1/um)
    P = np.sqrt((fx**2)+(fy**2))
    P = (P < (NA/lamda)).astype(float) # This is a filter transfer function for all the frequencies that fulfill numerical aperture requirement
    I = fftshift(fft2(mask))           #Fourier transfor of the mask shape.
    I = ifft2(P*I)                     # multiplication in freq domain = convolution of the mask with the NA filter + fft inversion back to space domain
    I = np.real(I*np.conj(I))          # square of the norm of the complex intensity in freq domain
    #I = I/np.max(I)                    # normalisation (is this necessay?)
    D_exp = 20*t_exp_sec #[power/cm^2]
    phys_I = I*D_exp #Dose [mJ/cm^2]
    aerial = (phys_I > TH).astype(float)
    return phys_I, mask, aerial

#this function will help visualize the mask as intended and as developed
#it also print the size of the openings identified along y_pixel for the original design
def plot_mask(mask,aerial,y_pixel):
    print('The dimensions of the mask are', round(Lx,2), 'x', round(Ly,2), 'um')
    print('Analysis of opening size expected accroding to mask pixelization:')
    calculate_apertures(mask,y_pixel,0.5)
    # Plots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 16))
    ax1.imshow(mask, cmap='gray', origin='lower',  extent=(-Lx/2, Lx/2, -Ly/2, Ly/2))
    ax1.set_title('Mask image')
    ax1.set_xlabel('X [um]')
    ax1.set_ylabel('Y [um]')
    ax2.imshow(aerial, cmap='gray', origin='lower', extent=(-Lx/2, Lx/2, -Ly/2, Ly/2))
    ax2.set_title('Aerial image')
    ax2.set_xlabel('X [um]')
    ax2.set_ylabel('Y [um]')
    plt.tight_layout()
      
# This function calculates and plots the dose profile on x and y plane
def plot_func(lamda,t_exp_sec):
    I, mask, aerial = update_3D(lamda,N, t_exp_sec)
    x = np.linspace(-margin_factor/2*Lxy+dx/2,margin_factor/2*Lxy-dx/2,N)
    y = np.linspace(-margin_factor/2*Lxy+dy/2,margin_factor/2*Lxy-dy/2,N)
    x, y = np.meshgrid(x, y)
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(x, y, I, cmap='jet')   
    ax.set_xlabel('X [um]')
    ax.set_ylabel('Y [um]')
    ax.set_zlabel('Local dose (mJ/cm^2)', labelpad=0.2)
    ax.set_zlim(0, 200)
    ax.set_title('Dose profile for nominal dose of '+ str(20*t_exp_sec) + r' $mJ/cm^2$')
    plt.show()
    return I, mask, aerial


def calculate_apertures(I,y_pixel,TH): 
    #this function takes a slide of I over column index y_pixel
    #and then prints the sizes of the continuous sections where I is greater than TH
    #it can be used to identify and measure zones where exposure overcomes full development threshold
    #or where the mask was designed to be really open.
    arr = np.where((I[y_pixel,:] >= TH))[0]  #list of index where the intensity is higher than the threshold
    output = []
    temp = []
    dist = []
    if(len(arr) != 0):
        for i, val in enumerate(arr[:-1]):
            temp.append(val)
            if arr[i+1] != val +1:
                output.append(temp)
                temp = []
        temp.append(arr[-1])
        output.append(temp)
        for j in range(len(output)):
            x_left_out=output[j][0]-1
            x_right_out=output[j][len(output[j])-1]+1
            I_left_in=I[y_pixel,output[j][0]]
            I_left_out=I[y_pixel,x_left_out]
            I_right_in=I[y_pixel,output[j][len(output[j])-1]]
            I_right_out=I[y_pixel,x_right_out]               
            dist.append((x_right_out-x_left_out-(TH-I_right_out)/(I_right_in-I_right_out)-(TH-I_left_out)/(I_left_in-I_left_out))*dx)
        if len(dist)>1:
            print('{:d} openings have been identified along this y_pixel line'.format(len(dist)))
            print('The width of the openings are :')
            for d in dist:
                print('{:.3f} [um]'.format(d))
        elif len(dist)==1:
            print('1 opening has been identified along this y_pixel line'.format(len(dist)))
            print('The width of the opening is :')
            for d in dist:
                print('{:.3f} [um]'.format(d))
    else:
        print('No openings have been identified along this y_pixel line')


def dev_thickness(dose):
    result=dev_curve(dose)
    result[dose>72]=2.5 #artifact to avoid negative values due to polynomial approximation
    return result

def replace_negative_with_zero(arr):
    arr[arr < 0] = 0
    return arr

def remain_thickness(init_thickness,dose):
    remain_thick=init_thickness-dev_thickness(dose)
    remain_thick=replace_negative_with_zero(remain_thick)
    return remain_thick
    
def plot_func_2D(I, y_pixel,t_exp_sec):#(lamda, y_pixel, t_exp_sec):
    #I, mask, aerial = update_3D(lamda,N, t_exp_sec)
    x = np.linspace(-margin_factor/2*Lxy+dx/2,margin_factor/2*Lxy-dx/2,N)
    D_exp = 20*t_exp_sec #[power/cm^2]
    plt.plot(x, I[y_pixel,:], label='Dose function (mJ/cm^2)')
    plt.plot(x, ~(I[y_pixel,:] >= TH)*D_exp, label='Development function')
    plt.plot(x, remain_thickness(init_th,I[y_pixel,:])/init_th*D_exp, color='gray', linestyle='--', label='Resit profile')
    plt.axhline(TH, color='red', linestyle='--', label='Threshold for development')
    plt.legend(loc='upper right', bbox_to_anchor=(1.6, 1))
    print('The dose is', D_exp, '[mJ/cm^2]')
    calculate_apertures(I,y_pixel,TH)
    
    y_aux = np.linspace(-margin_factor/2*Lxy+dy/2,margin_factor/2*Lxy-dy/2,N)
    print('You are at position y =', round(y_aux[y_pixel],3), ' [um]')
    
    plt.xlabel('X [um]')
    plt.ylabel('Z')
    plt.title('Dose and development function at y_pixel='+ str(y_pixel))
    plt.show()


def plot_all(lamda, y_pixel, t_exp_sec):
    I, mask, aerial = plot_func(lamda, t_exp_sec)
    plot_func_2D(I,y_pixel,t_exp_sec)#(lamda, y_pixel, t_exp_sec)
    plot_mask(mask,aerial,y_pixel) 

#buttons creation
Source_Dropdown = widgets.Dropdown(
       options=['F2 laser', 'ArF excimer laser', 'Hg/Xe arc lamp (DUV)', 'Hg arc lamp (I-line)', 'Hg arc lamp (H-line)', 'Hg arc lamp (G-line)'],
       description='Light source:')
Select_button = widgets.Button(description='Select')
outt = widgets.Output()


def on_butt_clicked(b):
    with outt:
        global prev_value
        global lamda
        clear_output()
        if(Source_Dropdown.value != prev_value):
            if(Source_Dropdown.value == 'F2 laser'):
                prev_value = 'F2 laser'
                lamda = 0.157
            elif(Source_Dropdown.value == 'ArF excimer laser'):
                prev_value = 'ArF excimer laser'
                lamda = 0.193
            elif(Source_Dropdown.value == 'Hg/Xe arc lamp (DUV)'):
                prev_value = 'Hg/Xe arc lamp (DUV)'
                lamda = 0.248 
            elif(Source_Dropdown.value == 'Hg arc lamp (I-line)'):
                lamda = 0.365 
            elif(Source_Dropdown.value == 'Hg arc lamp (H-line)'):
                lamda = 0.405 
            elif(Source_Dropdown.value == 'Hg arc lamp (G-line)'):
                lamda = 0.436 
        print('Wavelength = ', lamda, 'um') 
        interact(plot_all,  lamda=fixed(lamda), y_pixel = (0, N, 1), t_exp_sec = (0, 10,  0.25))


Select_button.on_click(on_butt_clicked)
widgets.VBox([Source_Dropdown,Select_button,outt])

## __This is the end of the photolithography notebook.__